# IMPORTS AND USER INPUTS

In [1]:
import numpy as np
import math
import pandas as pd
from scipy.stats import norm

filePath = input("Enter the filename: ")
df = pd.read_csv(filePath, sep = "\t", lineterminator = '\n', header = None)
        
sample_need = input("Do you wish to input a sample (Y/N): ").lower()
sample = ""
upd_sample = ""
if(sample_need=="y"):
    sample = (input("Please enter the sample in the {a,b,c,d} format: "))
    upd_sample = sample[1:-1].split(",") 
#{rain,hot,high,weak}

Enter the filename: data2.txt
Do you wish to input a sample (Y/N): n


# FUNCTIONS

In [2]:
def feature_check(list1):
    categ = True
    for i in (list1):
        if(i.isalpha() == False):
            categ = False
            break
    return categ

def demo_nb(X):
    categ = feature_check(df.iloc[0,:-1].values.tolist()) and feature_check(X)
    if(categ == False):
        print("This branch is for categorical dataset and sample!")
    else:
        items_list = {}
        for i in range(df.shape[1]-1):
            unique_labels12 = np.unique(df.iloc[:,i].values).tolist()
            items_list[i] = unique_labels12
            
        prob_dict = {}
        (unique_labels,counts) = np.unique(df.iloc[:,-1].values, return_counts = True)
        for i in range(len(counts)):
            prob_dict[unique_labels[i]] = counts[i]/df.shape[0]

        final_prob = {}
        for i in prob_dict:
            temp_dict1 = {}
            for j in range((df.shape[1]-1)):
                temp_dict2 = {}
                temp_list = (np.unique(df.iloc[:,j].values))
                for k in temp_list:
                    prob = len(df.loc[(df[j]==k) & (df[df.shape[1]-1]==i)])/len(df.loc[df[df.shape[1]-1]==i])
                    temp_dict2[k] = prob
                temp_dict1[j] = temp_dict2
            final_prob[i] = temp_dict1

        X_map = []
        for i in X:
            for j in items_list:
                if(i in items_list[j]):
                    X_map.append(j)
                    break

        for i in prob_dict:
            print("Probability of X belonging to Class "+str(i)+":")
            prob = prob_dict[i]
            for j in range(len(X)):
                item = X[j]
                item_index = X_map[j]
                prob *= final_prob[i][item_index][item]
            print(prob)

# IMPLEMENTATION

In [3]:
if(sample_need == "y"):
    demo_nb(upd_sample)
else:

    with open(filePath) as line1:
        lines=[line.split("\t") for line in line1]

    data = np.asarray(lines)
    sample_data = data[0]
    categorical_cols = []

    for col in range(sample_data.size):
        feature = sample_data[col]
        if feature.isalpha():
            ft_array,indices = np.unique(data[:,col],return_inverse = True)
            data[:,col] = indices.astype(np.float)
            categorical_cols.append(col)


    # Easier to leave categorical data out for normalization with matrix!
    dataset = np.matrix(data[:,:-1],dtype=float,copy=False)
    for col in range(dataset.shape[1]):
        if col not in categorical_cols:
            temp = dataset[:,col]
            dataset[:,col] = (temp - temp.min()) / (temp.max() - temp.min())


    truth_labels = np.asarray(data[:,-1],dtype = int)
    data_subset = np.array_split(dataset,10)
    truth_labels_subset = np.array_split(truth_labels,10)


    accuracy_average = 0
    precision_average = 0
    recall_average = 0
    f1_Measure_average = 0


    def performance_metric(actual, prediction):

        a = 0
        b = 0
        c = 0
        d = 0

        accuracy = 0
        precision = 0
        recall = 0
        f1_measure = 0

        for i in range(len(prediction)):
            if actual[i] == 1 and prediction[i] == 1:
                a += 1
            elif actual[i] == 1 and prediction[i] == 0:
                b += 1
            elif actual[i] == 0 and prediction[i] == 1:
                c += 1
            elif actual[i] == 0 and prediction[i] == 0:
                d += 1

        accuracy += (float(a+d)/(a+b+c+d))

        if(a+c != 0):
            precision += (float(a)/(a+c))

        if(a+b != 0):
            recall += (float(a)/(a+b))

        f1_measure += (float(2*a)/((2*a)+b+c))

        return accuracy, precision, recall, f1_measure





    for subs in range(10):
        # We have combined all the training data  using vstack. Not exactly all the training data, we have select 90% training data.
        # The rest 10% are in the testing_data

        training_data = np.asarray(np.vstack([x for i,x in enumerate(data_subset) if i != subs]))
        train_label = np.asarray(np.concatenate([x for i,x in enumerate(truth_labels_subset) if i != subs]))

        testing_data = np.asarray(data_subset[subs])
        actual_label = np.asarray(truth_labels_subset[subs])

        # Over here we are fetching just the continuous data so that we can get the Gaussian Probability here
        continuous_traindata = np.delete(training_data,categorical_cols,axis=1)
        continuous_testdata = np.delete(testing_data,categorical_cols,axis=1)


        # Continuous Data Gaussian
        mean_0 = np.mean(continuous_traindata[train_label == 0.0],axis=0)
        std_0 = np.std(continuous_traindata[train_label == 0.0], axis=0)

        c0_result = norm.pdf(continuous_testdata, mean_0, std_0)

        mean_1 = np.mean(continuous_traindata[train_label == 1.0],axis=0)
        std_1 = np.std(continuous_traindata[train_label == 1.0], axis=0)

        c1_result = norm.pdf(continuous_testdata, mean_1, std_1)


        final_0_resultpdf = []
        final_1_resultpdf = []

        for i2 in range(len(testing_data)):
            final_0_resultpdf.append(np.prod(c0_result[i2]))
            final_1_resultpdf.append(np.prod(c1_result[i2]))


        categorical_0 =  np.ones((len(testing_data)))
        categorical_1 = np.ones((len(testing_data)))

        for i in range(len(categorical_cols)):

            tcat_data = training_data[:,categorical_cols[i]]
            testcat_data = testing_data[:,categorical_cols[i]]

            tr0_categorical = tcat_data[train_label == 0.0]
            tr1_categorical = tcat_data[train_label == 1.0]

            pb_0 = len(tr0_categorical)/len(tcat_data)
            pb_1 = len(tr1_categorical)/len(tcat_data)

            for j in range(len(testcat_data)):

                pbx_0 = len(tr0_categorical[np.where((tr0_categorical ==testcat_data[j]))])/len(tr0_categorical)
                pbx_1 = len(tr1_categorical[np.where((tr1_categorical ==testcat_data[j]))])/len(tr1_categorical)

                if pbx_0 == 0 or pbx_0 == 1:
                    pbx_0 = (len(tr0_categorical[np.where((tr0_categorical ==testcat_data[j]))]  )) /(len(tr0_categorical) + len(set(tcat_data))) #Laplace


                if pbx_1 == 0 or pbx_1 == 1:
                    pbx_1 = (len(tr1_categorical[np.where((tr1_categorical ==testcat_data[j]))])) /(len(tr1_categorical) + len(set(tcat_data)))

                pb0_x = (pbx_0 * pb_0)  / len(tcat_data[tcat_data==testcat_data[j]])/len(tcat_data)
                pb1_x = (pbx_1 * pb_1) / len(tcat_data[tcat_data==testcat_data[j]])/len(tcat_data)

                categorical_0[j] *= (pb0_x)

                categorical_1[j] *= (pb1_x)

        #now multiply zero and one prob of continuous and cat data


        predictions = []

        for k in range(len(testing_data)):
            # Now we are combining the categorical and continuous variable probabilities!!!

            if (categorical_0[k] * final_0_resultpdf[k]) > (categorical_1[k] * final_1_resultpdf[k]):
                predictions.append(0)
            else:
                predictions.append(1)

        accuracy, precision, recall, f1_measure = performance_metric(actual_label, predictions)

        print("Accuracy -> ",accuracy)
        accuracy_average += accuracy

        print("Precision -> ",precision)
        precision_average += precision

        print("Recall -> ",recall)
        recall_average += recall

        print("F1_measure -> ",f1_measure)
        f1_Measure_average += f1_measure
        print("----------------------------------------------------------------")

    print("Average Accuracy -> ",accuracy_average/10)
    print("Average Precision -> ",precision_average/10)
    print("Average Recall -> ",recall_average/10)
    print("Average F_measure -> ",f1_Measure_average/10)

Accuracy ->  0.6595744680851063
Precision ->  0.6
Recall ->  0.7142857142857143
F1_measure ->  0.6521739130434783
----------------------------------------------------------------
Accuracy ->  0.7446808510638298
Precision ->  0.5238095238095238
Recall ->  0.8461538461538461
F1_measure ->  0.6470588235294118
----------------------------------------------------------------
Accuracy ->  0.782608695652174
Precision ->  0.7647058823529411
Recall ->  0.6842105263157895
F1_measure ->  0.7222222222222222
----------------------------------------------------------------
Accuracy ->  0.6739130434782609
Precision ->  0.5
Recall ->  0.6
F1_measure ->  0.5454545454545454
----------------------------------------------------------------
Accuracy ->  0.6304347826086957
Precision ->  0.6
Recall ->  0.45
F1_measure ->  0.5142857142857142
----------------------------------------------------------------
Accuracy ->  0.6304347826086957
Precision ->  0.5263157894736842
Recall ->  0.5555555555555556
F1_measure